In [1]:
import os
import time
import pyproj
import numpy as np
import pandas as pd
import configparser
import matplotlib.pyplot as plt

In [15]:
# e.g. DATA_PATH = "/home/user/data/Google/training/""
DATA_PATH = "/home/derek/datasets/google-challenge/train/"

In [16]:
# used for receiver clock bias estimation

def least_squares(gt_ecef,data,rb):
    """
    input(s)
        gt_ecef: state estimate
        data: satellite and psuedorange dataframe
        rb: new receiver clock bias
    output(s)
        rb: new receiver clock bias
    """
    numSats = len(data)

    sat_pos = np.hstack((data["xSatPosM"].to_numpy().reshape(-1,1),
                         data["ySatPosM"].to_numpy().reshape(-1,1),
                         data["zSatPosM"].to_numpy().reshape(-1,1)))
    gt_pos = np.tile(gt_ecef,(sat_pos.shape[0],1))

    gt_psuedoranges = np.linalg.norm(gt_pos - sat_pos, axis = 1)

    G = np.ones((numSats,4))
    G[:,:3] = np.divide(gt_pos - sat_pos,gt_psuedoranges.reshape(-1,1))

    W = np.diag(1./data["correctedPrM"])

    rho_diff = data["correctedPrM"].to_numpy() \
             + data["satClkBiasM"].to_numpy() \
             - gt_psuedoranges \
             - rb

    rho_diff = rho_diff.reshape(-1,1)

    delta = np.linalg.pinv(W.dot(G)).dot(W).dot(rho_diff)
    rb_new = rb + delta[3,0]
    return rb_new

In [22]:
# used to save plots

def prep_logs(trace_name, phone_type):
    """create log directories if they don't yet exist

    """
    repo_dir = os.path.split(os.getcwd())[0]

    log_dir = os.path.join(repo_dir,"log","residual_plots")

    # create log data directory if it doesn't yet exist
    if not os.path.isdir(log_dir):
        try:
            os.makedirs(log_dir)
        except OSError as e:
            print("e: ",e)
            sys.exit(1)

    return log_dir

In [23]:
def calc_residuals(trace_name, phone_type, verbose = False):
    """Calculate residuals

    Parameters
    ----------
    trace_name : string
        Provided name for trace, e.g., "2020-05-14-US-MTV-1".
    phone_type : string
        Type of phone used to record data, e.g., "Pixel4", "Pixel4XL",
        "Pixel4XLModded", "Mi8".

    """

    print("Analyzing trace ",trace_name,phone_type,"...")

    ####################################################################
    # PARAMETERS
    ####################################################################

    # how many measurement rows to calculate
    # -1 means keep all measurements
    MEASUREMENTS_HEAD = -1

    ####################################################################
    # END PARAMETERS
    ####################################################################

    data_path = DATA_PATH

    # load measurements
    input_filepath = os.path.join(data_path, trace_name, phone_type,
                                  phone_type + "_derived.csv")
    measurements = pd.read_csv(input_filepath)

    # crop measurements according to MEASUREMENTS_HEAD parameter
    if MEASUREMENTS_HEAD != -1:
        measurements = measurements.head(MEASUREMENTS_HEAD)

    # calculate corrected psuedorange
    measurements["correctedPrM"] = measurements["rawPrM"] \
                                 - measurements["isrbM"] \
                                 - measurements["ionoDelayM"] \
                                 - measurements["tropoDelayM"]
    # add SvName
    measurements["SvName"] = measurements["constellationType"].replace(
                             [1,3,4,5,6],["G","R","Q","B","E"]) \
                           + measurements["svid"].astype(str)

    # load ground truth
    gt_filepath = os.path.join(data_path, trace_name, phone_type,
                                  "ground_truth.csv")
    gt = pd.read_csv(gt_filepath)
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    x, y, z = pyproj.transform(lla, ecef, gt["lngDeg"], gt["latDeg"],
                               gt["heightAboveWgs84EllipsoidM"],
                               radians=False)
    gt["ecefxM"] = x
    gt["ecefyM"] = y
    gt["ecefzM"] = z

    residual_data = {}

    log_dir = prep_logs(trace_name, phone_type)

    e = 0

    for _,data in measurements.groupby("millisSinceGpsEpoch",as_index=False):

        if e % 1000 == 0 and verbose:
            print("e: ",e)

        epoch_gt = gt.iloc[abs(gt["millisSinceGpsEpoch"]
                 - data.loc[data.first_valid_index(),
                            "millisSinceGpsEpoch"]).argsort()[0]]
        gt_ecef = np.array([epoch_gt["ecefxM"],
                            epoch_gt["ecefyM"],
                            epoch_gt["ecefzM"]]).T

        # calculate "truth psuedorange"
        sat_pos = np.hstack((data["xSatPosM"].to_numpy().reshape(-1,1),
                             data["ySatPosM"].to_numpy().reshape(-1,1),
                             data["zSatPosM"].to_numpy().reshape(-1,1)))
        gt_pos = np.tile(gt_ecef,(sat_pos.shape[0],1))

        gt_psuedoranges = np.linalg.norm(gt_pos - sat_pos, axis = 1)

        # calculate receiver clock bias
        rb = 0.0
        for ii in range(20):
            rb = least_squares(gt_ecef,data,rb)

        # calculate residual
        data.loc[:,"residual"] = data["correctedPrM"].to_numpy() \
                               + data["satClkBiasM"].to_numpy() \
                               - gt_psuedoranges \
                               - rb

        # recalculate receiver clock bias
        residual_mean = data["residual"].mean()
        residual_std = data["residual"].std()
        distance_from_mean = abs(data["residual"] - residual_mean)
        data_rb = data[distance_from_mean <= 3*residual_std]

        # calculate receiver clock bias
        rb = 0.0
        for ii in range(20):
            rb = least_squares(gt_ecef,data_rb,rb)

        # calculate residual
        data.loc[:,"residual"] = data["correctedPrM"].to_numpy() \
                               + data["satClkBiasM"].to_numpy() \
                               - gt_psuedoranges \
                               - rb

        # drop duplicates for residual plotting
        data.drop_duplicates(subset = ["SvName"],
                             inplace = True)

        # plot residual data
        for index, row in data.iterrows():
            elapsed_sec = (data.loc[data.first_valid_index(),"millisSinceGpsEpoch"] -
                           measurements.loc[0,"millisSinceGpsEpoch"]) / 1000.
            if row["SvName"] not in residual_data:
                residual_data[row["SvName"]] = [[elapsed_sec,
                                      row["residual"]]]
            else:
                residual_data[row["SvName"]].append([elapsed_sec,
                                          row["residual"]])
        e += 1

    residual_fig = plt.figure()
    for key, value in residual_data.items():
        value = np.array(value)
        plt.plot(value[:,0],value[:,1],label=key)

    plt.xlabel("time [s]")
    plt.ylabel("residiual [m]")
    plt.legend()


    plt_file = os.path.join(log_dir, trace_name + "-" \
             + phone_type + "-residual-" + str(MEASUREMENTS_HEAD) \
             + ".png")

    residual_fig.savefig(plt_file,
            format="png",
            bbox_inches="tight")

    plt.ylim(-100,100)
    plt_file = os.path.join(log_dir, trace_name + "-" \
             + phone_type + "-residual-" + str(MEASUREMENTS_HEAD) \
             + "-zoomed" + ".png")

    residual_fig.savefig(plt_file,
            format="png",
            bbox_inches="tight")

    plt.close(residual_fig)

In [24]:
def main(trace_list):

    total_time = 0.0

    for trace in trace_list:
        trace_name, phone_type = trace

        time0 = time.time()

        calc_residuals(trace_name, phone_type, True)

        trace_time = time.time() - time0
        total_time += trace_time
        print("trace analysis took ", trace_time, " sec.")
        print("total time of ", total_time/60., " minutes.")

In [ ]:
# get all trace options
trace_names = sorted(os.listdir(DATA_PATH))

# create a list of all traces with phone types
trace_list = []
for trace_name in trace_names:
    trace_path = os.path.join(DATA_PATH,trace_name)
    for phone_type in sorted(os.listdir(trace_path)):
        trace_list.append((trace_name,phone_type))

# trace_list = [
#               ('2020-05-14-US-MTV-1', 'Pixel4'),
#               ('2020-05-14-US-MTV-1', 'Pixel4XLModded'),
#               ('2020-05-14-US-MTV-2', 'Pixel4'),
#               ('2020-05-14-US-MTV-2', 'Pixel4XLModded'),
#               ('2020-05-21-US-MTV-1', 'Pixel4'),
#               ('2020-05-21-US-MTV-2', 'Pixel4'),
#               ('2020-05-21-US-MTV-2', 'Pixel4XL'),
#               ('2020-05-29-US-MTV-1', 'Pixel4'), # 12/12 faulty
#               ('2020-05-29-US-MTV-1', 'Pixel4XL'),
#               ('2020-05-29-US-MTV-1', 'Pixel4XLModded'),
#               ('2020-05-29-US-MTV-2', 'Pixel4'),
#               ('2020-05-29-US-MTV-2', 'Pixel4XL'),
#               ('2020-06-04-US-MTV-1', 'Pixel4'),
#               ('2020-06-04-US-MTV-1', 'Pixel4XL'),
#               ('2020-06-04-US-MTV-1', 'Pixel4XLModded'),
#               ('2020-06-05-US-MTV-1', 'Pixel4'),
#               ('2020-06-05-US-MTV-1', 'Pixel4XL'),
#               ('2020-06-05-US-MTV-1', 'Pixel4XLModded'),
#               ('2020-06-05-US-MTV-2', 'Pixel4'),
#               ('2020-06-05-US-MTV-2', 'Pixel4XL'),
#               ('2020-06-11-US-MTV-1', 'Pixel4'),
#               ('2020-06-11-US-MTV-1', 'Pixel4XL'),
#               ('2020-07-08-US-MTV-1', 'Pixel4'),
#               ('2020-07-08-US-MTV-1', 'Pixel4XL'),
#               ('2020-07-08-US-MTV-1', 'Pixel4XLModded'),
#               ('2020-07-17-US-MTV-1', 'Mi8'),
#               ('2020-07-17-US-MTV-2', 'Mi8'), # 28/28 faulty
#               ('2020-08-03-US-MTV-1', 'Mi8'),
#               ('2020-08-03-US-MTV-1', 'Pixel4'),
#               ('2020-08-06-US-MTV-2', 'Mi8'),
#               ('2020-08-06-US-MTV-2', 'Pixel4'),
#               ('2020-08-06-US-MTV-2', 'Pixel4XL'),
#               ('2020-09-04-US-SF-1', 'Mi8'), # 29/31 faulty
#               ('2020-09-04-US-SF-1', 'Pixel4'),
#               ('2020-09-04-US-SF-1', 'Pixel4XL'),
#               ('2020-09-04-US-SF-2', 'Mi8'),
#               ('2020-09-04-US-SF-2', 'Pixel4'), # WARN 17/40, 18/42
#               ('2020-09-04-US-SF-2', 'Pixel4XL'),
#               ('2021-01-04-US-RWC-1', 'Pixel4'),
#               ('2021-01-04-US-RWC-1', 'Pixel4Modded'),
#               ('2021-01-04-US-RWC-1', 'Pixel4XL'),
#               ('2021-01-04-US-RWC-1', 'Pixel5'),
#               ('2021-01-04-US-RWC-2', 'Pixel4'),
#               ('2021-01-04-US-RWC-2', 'Pixel4Modded'),
#               ('2021-01-04-US-RWC-2', 'Pixel4XL'),
#               ('2021-01-04-US-RWC-2', 'Pixel5'),
#               ('2021-01-05-US-SVL-1', 'Mi8'),
#               ('2021-01-05-US-SVL-1', 'Pixel4'),
#               ('2021-01-05-US-SVL-1', 'Pixel4XL'), # 6/6 faulty
#               ('2021-01-05-US-SVL-1', 'Pixel5'), # 10/10 faulty
#               ('2021-01-05-US-SVL-2', 'Pixel4'),
#               ('2021-01-05-US-SVL-2', 'Pixel4Modded'),
#               ('2021-01-05-US-SVL-2', 'Pixel4XL'),
#               ('2021-03-10-US-SVL-1', 'Pixel4XL'),
#               ('2021-03-10-US-SVL-1', 'SamsungS20Ultra'),
#               ('2021-04-15-US-MTV-1', 'Pixel4'), # WARN 10/21 faulty
#               ('2021-04-15-US-MTV-1', 'Pixel4Modded'),
#               ('2021-04-15-US-MTV-1', 'Pixel5'),
#               ('2021-04-15-US-MTV-1', 'SamsungS20Ultra'),
#               ('2021-04-22-US-SJC-1', 'Pixel4'), # 6/7 faulty
#               ('2021-04-22-US-SJC-1', 'SamsungS20Ultra'),
#               ('2021-04-26-US-SVL-1', 'Mi8'), # WARN 25/33 faulty
#               ('2021-04-26-US-SVL-1', 'Pixel5'),
#               ('2021-04-28-US-MTV-1', 'Pixel4'),
#               ('2021-04-28-US-MTV-1', 'Pixel5'),
#               ('2021-04-28-US-MTV-1', 'SamsungS20Ultra'),
#               ('2021-04-28-US-SJC-1', 'Pixel4'), # 26/26 faulty
#               ('2021-04-28-US-SJC-1', 'SamsungS20Ultra'),
#               ('2021-04-29-US-MTV-1', 'Pixel4'),
#               ('2021-04-29-US-MTV-1', 'Pixel5'),
#               ('2021-04-29-US-MTV-1', 'SamsungS20Ultra'), # 28/28 faulty
#               ('2021-04-29-US-SJC-2', 'Pixel4'), # 6/6 faulty
#               ('2021-04-29-US-SJC-2', 'SamsungS20Ultra'),
# ]

main(trace_list)

Analyzing trace  2020-05-14-US-MTV-1 Pixel4 ...


/tmp/ipykernel_7807/2959084278.py:55: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y, z = pyproj.transform(lla, ecef, gt["lngDeg"], gt["latDeg"],


e:  0
e:  1000
